# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [1]:
# Loading libraries

#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [2]:
altright_wikipage = 'https://en.wikipedia.org/wiki/Alt-right'
altright_wikiContentRequest = requests.get(altright_wikipage)
altright_wikiContentSoup = bs4.BeautifulSoup(altright_wikiContentRequest.text, 'html.parser')
print(altright_wikiContentSoup.text[:200])





Alt-right - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgM


In [3]:
altright_content_analysis_save = 'wikipedia_content_analysis.html'

with open(altright_content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(altright_wikiContentSoup.text)
    
altright_contentPTags = altright_wikiContentSoup.body.findAll('p')

In [4]:
altright_contentParagraphs = []
for pTag in altright_contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    altright_contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
altright_contentParagraphsDF = pandas.DataFrame({'paragraph_text' : altright_contentParagraphs})
print(altright_contentParagraphsDF)

                                        paragraph_text
0                                                   \n
1                                                   \n
2    The alt-right, an abbreviation of alternative ...
3    In 2010, the American white nationalist Richar...
4    The alt-right is a biologically racist movemen...
..                                                 ...
123                              Informational notes\n
124                                        Citations\n
125                                     Bibliography\n
126                                 Further reading \n
127  Cateogry:Opposition to Islam in the United Sta...

[128 rows x 1 columns]


1. Paragraphs where Richard Spencer is mentioned

In [5]:
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Spencer')]
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Spencer')][:5]

,paragraph_text
3,"In 2010, the American white nationalist Richar..."
6,"The term ""alt-right"" is a neologism first used..."
18,"According to Hawley, the alt-right began in 20..."
19,"After The American Conservative fired Spencer,..."
20,AlternativeRight.com consisted primarily of sh...


2. Paragraphs where Charlottesville is mentioned

In [6]:
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Charlo*')]
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Charlo*')][:5]

,paragraph_text
3,"In 2010, the American white nationalist Richar..."
38,"In August 2017, the Unite the Right rally took..."
39,Alt-rightists who attended the rally often fou...
97,"In February 2018, the Southern Poverty Law Cen..."
105,The ramming occurred at a pedestrian mall at W...


3. Paragraphs where Nazi* or fasci* is mentioned

In [7]:
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Nazi*|fasci*', regex = True)]
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Nazi*|fasci*', regex = True)][:5]

,paragraph_text
3,"In 2010, the American white nationalist Richar..."
4,The alt-right is a biologically racist movemen...
5,The alt-right distinguished itself from earlie...
6,"The term ""alt-right"" is a neologism first used..."
9,The anti-fascist researcher Matthew N. Lyons d...


4. Paragraphs where Trump is mentioned

In [8]:
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Trump')]
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Trump')][:5]

,paragraph_text
3,"In 2010, the American white nationalist Richar..."
27,"On September 9, 2016, several figures of the a..."
28,"In June 2015, New York billionaire businessman..."
29,The alt-right was exceedingly vocal in support...
30,"As noted by Hawley, ""the Alt-Right is most def..."


5. Paragraphs where Breitbart is mentioned

In [9]:
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Breitbart')]
altright_contentParagraphsDF[altright_contentParagraphsDF.paragraph_text.str.contains('Breitbart')][:5]

,paragraph_text
3,"In 2010, the American white nationalist Richar..."
24,Many press sources described Breitbart News as...
25,"In March 2016, Breitbart writers Allum Bokhari..."
32,Breitbart's Bannon had been one of Trump's mos...
39,Alt-rightists who attended the rally often fou...



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>

In [10]:
wikipedia_base_url = 'https://en.wikipedia.org'

altright_otherPageURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(altright_contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        altright_otherPageURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(altright_otherPageURLS)

[('https://en.wikipedia.org/wiki/Far-right_politics', 2, 'far-right'), ('https://en.wikipedia.org/wiki/White_nationalism', 2, 'white nationalist'), ('https://en.wikipedia.org/wiki/Internet_activism', 2, 'online phenomenon'), ('https://en.wikipedia.org/wiki/Antisemitism', 2, 'antisemitism'), ('https://en.wikipedia.org/wiki/Richard_B._Spencer', 3, 'Richard B. Spencer'), ('https://en.wikipedia.org/wiki/Paleoconservatism', 3, 'paleoconservatism'), ('https://en.wikipedia.org/wiki/Dark_Enlightenment', 3, 'Dark Enlightenment'), ('https://en.wikipedia.org/wiki/Nouvelle_Droite', 3, 'Nouvelle Droite'), ('https://en.wikipedia.org/wiki//pol/', 3, '/pol/'), ('https://en.wikipedia.org/wiki/4chan', 3, '4chan'), ('https://en.wikipedia.org/wiki/Traditionalist_Worker_Party', 3, 'Traditionalist Worker Party'), ('https://en.wikipedia.org/wiki/Gamergate_controversy', 3, 'Gamergate controversy'), ('https://en.wikipedia.org/wiki/Steve_Bannon', 3, 'Steve Bannon'), ('https://en.wikipedia.org/wiki/Breitbart_New

In [11]:
altright_contentParagraphsDF['source'] = [altright_wikipage] * len(altright_contentParagraphsDF['paragraph_text'])
altright_contentParagraphsDF['paragraph_number'] = range(len(altright_contentParagraphsDF['paragraph_text']))

altright_contentParagraphsDF[:5]

,paragraph_text,source,paragraph_number
0,\n,https://en.wikipedia.org/wiki/Alt-right,0
1,\n,https://en.wikipedia.org/wiki/Alt-right,1
2,"The alt-right, an abbreviation of alternative ...",https://en.wikipedia.org/wiki/Alt-right,2
3,"In 2010, the American white nationalist Richar...",https://en.wikipedia.org/wiki/Alt-right,3
4,The alt-right is a biologically racist movemen...,https://en.wikipedia.org/wiki/Alt-right,4


In [12]:
altright_contentParagraphsDF['source_paragraph_number'] = [None] * len(altright_contentParagraphsDF['paragraph_text'])
altright_contentParagraphsDF['source_paragraph_text'] = [None] * len(altright_contentParagraphsDF['paragraph_text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph_number' : [], 'paragraph_text' : [], 'source_paragraph_number' : [],  'source_paragraph_text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph_text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph_number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source_paragraph_number'].append(sourceParNum)
        parsDict['source_paragraph_text'].append(sourceText)
    return pandas.DataFrame(parsDict)

for urlTuple in altright_otherPageURLS[:5]:
    #ignore_index means the indices will not be reset after each append
    altright_contentParagraphsDF = altright_contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)

altright_contentParagraphsDF[:5]

,paragraph_number,paragraph_text,source,source_paragraph_number,source_paragraph_text
0,0,\n,https://en.wikipedia.org/wiki/Alt-right,None,None
1,1,\n,https://en.wikipedia.org/wiki/Alt-right,None,None
2,2,"The alt-right, an abbreviation of alternative ...",https://en.wikipedia.org/wiki/Alt-right,None,None
3,3,"In 2010, the American white nationalist Richar...",https://en.wikipedia.org/wiki/Alt-right,None,None
4,4,The alt-right is a biologically racist movemen...,https://en.wikipedia.org/wiki/Alt-right,None,None


In [35]:
def get_pageURLs(start_wikiURL):
    otherPageURLs = []
    start_wikiContentRequest = requests.get(start_wikiURL)
    start_wikiContentSoup = bs4.BeautifulSoup(start_wikiContentRequest.text, 'html.parser')
    start_contentPTags = start_wikiContentSoup.body.findAll('p')
    for pTag in start_contentPTags:
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for aTag in tagLinks:
            relurl = aTag.get('href')
            full_url = urllib.parse.urljoin(wikipedia_base_url, relurl)
            if full_url not in otherPageURLs:
                otherPageURLs.append(full_url)
                    
    return otherPageURLs

def rec_spider(start_wikiURL, stop_level):    
    global ref_list
    ref_list = []
    
    for i in range(stop_level):
        if len(ref_list) == 0:
            ref_list = get_pageURLs(start_wikiURL)
            new_URLs = ref_list
        else:
            i_otherPageURLs = []
            for x in new_URLs:
                i_otherPageURLs.extend(get_pageURLs(x))
            unique_i_otherPageURLs = [] 
            [unique_i_otherPageURLs.append(x) for x in i_otherPageURLs if x not in unique_i_otherPageURLs] 
            new_URLs = [x for x in unique_i_otherPageURLs if x not in ref_list]
            ref_list.extend(new_URLs)
            
    return ref_list     

In [37]:
altright_reclist = rec_spider(altright_wikipage,2)
len(altright_reclist)

34978


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


The cell below creates a txt file from a Vox article. 

In [38]:
kcrenshaw_content_analysis_save = 'kcrenshaw_content_analysis.txt'
kcrenshaw_wpage = 'https://www.vox.com/the-highlight/2019/5/20/18542843/intersectionality-conservatism-law-race-gender-discrimination'

kcrenshaw_ContentRequest = requests.get(kcrenshaw_wpage)
kcrenshaw_ContentSoup = bs4.BeautifulSoup(kcrenshaw_ContentRequest.text, 'html.parser')

with open(kcrenshaw_content_analysis_save, mode='w', encoding='utf-8') as f:
    kcrenshaw_contentPTags = kcrenshaw_ContentSoup.body.findAll('p')
    kcrenshaw_contentParagraphs = []
    for pTag in kcrenshaw_contentPTags:
        f.write(re.sub(r'\[\d+\]', '', pTag.text) + '\n')

Code to read the above `txt` file into a pandas dataframe:

In [39]:
with open(kcrenshaw_content_analysis_save, mode = 'r', encoding = 'utf-8') as f:
    kcrenshaw_text = f.read().split('\n')
    
kcrenshaw_df = pandas.DataFrame({'text' : kcrenshaw_text})
kcrenshaw_df[:5]

,text
0,When Kimberlé Crenshaw coined the term 30 year...
1,There may not be a word in American conservati...
2,"To many conservatives, intersectionality means..."
3,This is a highly unusual level of disdain for ...
4,In my conversations with right-wing critics of...
